In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint
m = wt.watertap_setup(dynamic=False)


wt.case_study_trains.train = {"case_study": "solaire",
                             "reference": "nawi",
                             "scenario": "baseline"}

wt.case_study_trains.source_water = {"case_study": "solaire", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": ["solaire_graywater", "solaire_blackwater", "solaire_stormwater"]}

m = wt.case_study_trains.get_case_study(m=m) # flow is set as case study flow unless defined.



solaire

------- Adding Unit Processes -------
gray_black
stormwater
water_pumping_station
mbr_nitrification
ozone_aop
uv_irradiation
media_filtration
treated_storage
irrigation
surface_discharge
-------------------------------------

adding source
adding source
adding source
adding splitter

Connecting unit processes...


In [2]:
wt.display.show_train2(model_name=m)

In [3]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m=m, print_model_results="summary", skip_small=True)

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


GRAY BLACK:

	total cap investment: 0.01873
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00025

STORMWATER:

	total cap investment: 0.00739
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.0001

WATER PUMPING STATION:

	total cap investment: 0.00282
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 4e-05

MBR NITRIFICATION:

	total cap investment: 0.09831
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00131

OZONE AOP:

	total cap investment: 0.29952
	cat and chem cost: 0.00015
	electricity cost: 0.00014
	total fixed op cost: 0.00399

UV IRRADIATION:

	total cap investment: 0.02709
	cat and chem cost: 0.0
	electricity cost: 0.00039
	total fixed op cost: 0.00036

MEDI

In [4]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m=m, case_study=wt.case_study_trains.source_water["case_study"], 
                                scenario=wt.case_study_trains.source_water["scenario"])